In [ ]:
import pandas as pd
import json
import requests

In [ ]:
from config import BEA_API
from county_codes import stco_fips

In [ ]:
base_url = f'https://apps.bea.gov/api/data/?UserID={BEA_API}'

### User variables

In [ ]:
#Start & end years
year1 = '2018'
year0 = '2008'
years = f'{year0},{year1}'

#Nominal GDP for total GDP in 2018
nomGDP = 'CAGDP1'
nomGDP_lc = '3'

#Real GDP - 2012-chained GDP for change calcs
realGDP = 'CAGDP9'
realGDP_lc = '1'

geo = 'COUNTY'

#### Get Parameter Values for Regional Dataset (table IDs)

In [ ]:
#Get data from specific table
def get_data(tablename,linecode,geofips,year):    
    url = f'{base_url}&method=GetData&datasetname=Regional&TableName={tablename}&LineCode={linecode}&GeoFIPS={geofips}&Year={year}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['BEAAPI']['Results']['Data'])
    df = df[df['GeoFips'].isin(stco_fips)]
    return df

def var_cal(df):
    df['GDPrY0Y1'] = df[year1] - df[year0]
    df['GDPprY0Y1'] = ((df[year1]/df[year0])**(1/(int(year1)-int(year0))))-1
    return df

def clean_table(df):
    dff = df.drop(columns=[year0,year1]).rename(columns={'DataValue':'GDPY1'})
    for column_name in dff:
        dff.rename(columns={column_name:column_name.replace('Y0',year0[2:]).replace('Y1',year1[2:])},inplace=True)
    return dff

### County Table

In [ ]:
#Get nominal GDP for Year 1 (Current Year)
df_nom = get_data(nomGDP,nomGDP_lc,geo,year1)

In [ ]:
#Get real GDP to calculate change (Current - Init Year)
df_real = get_data(realGDP,realGDP_lc,geo,years)
df_real = df_real.pivot(index='GeoFips',columns='TimePeriod',values='DataValue')

In [ ]:
#Merge nominal and real gdp
gdp = pd.merge(df_real,df_nom,how='left',on='GeoFips').drop(columns=['CL_UNIT','Code','NoteRef','TimePeriod','GeoName','UNIT_MULT'],axis=1)

In [ ]:
#Clean data,
col = [year0,year1,'DataValue']
for i in col:
    gdp[i] = gdp[i].str.replace(',','').astype(float)
    gdp[i] = gdp[i]*1000

In [ ]:
gdp = var_cal(gdp)

In [ ]:
gdp

### Intermediate County Table - GDP

In [ ]:
co_gdp = clean_table(gdp)

In [ ]:
co_gdp = co_gdp.rename(columns={'GeoFips':'GEO_ID'})
co_gdp

In [ ]:
#to csv
co_gdp.to_csv('../output/county/co_gdp.csv')

### Make Subregion and Region Tables

In [ ]:
#Read in geography cross-walk file & clean id column for join
geo_xwalk = pd.read_excel('../data/31CR_CoxSub.xlsx')
geo_xwalk['stco'] = geo_xwalk['stco'].apply(lambda x: '{0:0>5}'.format(x))
geo_xwalk.head()

In [ ]:
#Merge xwalk file to reduce to region counties only & drop unneeded identifiers
df = pd.merge(gdp,geo_xwalk,how='left',left_on=['GeoFips'],right_on=['stco']).drop(columns=['st','co','stco_int','stco','stco_lbl','co_lbl','GDPrY0Y1','GDPprY0Y1'])
df.head()

#### Subregion

In [ ]:
sub_gdp = df.drop(columns=['reg','GeoFips']).groupby(['subreg']).sum()

In [ ]:
sub_gdp = var_cal(sub_gdp)
sub_gdp = clean_table(sub_gdp)
sub_gdp

In [ ]:
#to csv
sub_gdp.to_csv('../output/subregion/sub_gdp.csv')

#### Region

In [ ]:
reg_gdp = df.drop(columns=['subreg','GeoFips']).groupby(['reg']).sum()

In [ ]:
reg_gdp = var_cal(reg_gdp)
reg_gdp = clean_table(reg_gdp)
reg_gdp

In [ ]:
#to csv
reg_gdp.to_csv('../output/region/reg_gdp.csv')